In [7]:
import pandas as pd
from ChromosomeWindows import window

In [41]:
data = pd.DataFrame({'chrom': ['chr1']+['chr2']*10,
                    'start': list(range(10)) + [40],
                    'end': list(map(sum, zip(range(10), [5, 1]*5+[20]))) + [45],
                    'value': 'AAA',
                   'species': 7, 'run': 9, 'analysis' : 4})

data

,chrom,start,end,value,species,run,analysis
0,chr1,0,5,AAA,7,9,4
1,chr2,1,2,AAA,7,9,4
2,chr2,2,7,AAA,7,9,4
3,chr2,3,4,AAA,7,9,4
4,chr2,4,9,AAA,7,9,4
5,chr2,5,6,AAA,7,9,4
6,chr2,6,11,AAA,7,9,4
7,chr2,7,8,AAA,7,9,4
8,chr2,8,13,AAA,7,9,4
9,chr2,9,10,AAA,7,9,4


Make a function `interval_count` that is called on the intervals in windows of size 5. Note that the `window` decorator only handles a single chromosome so you always need to group your data by chromosome:

In [91]:
@window(size=5)
def interval_count(df):
    return len(df.index)

df = data.groupby('chrom').apply(interval_count)
df

start  end  interval_count
chrom                              
chr1  0      0    5               1
chr2  0      0    5               4
      1      5   10               7
      2     10   15               2
      3     15   20               0
      4     20   25               0
      5     25   30               0
      6     30   35               0
      7     35   40               0
      8     40   45               1

You can get rid of the extra index like this:

In [92]:
df.reset_index(drop=True, level=-1)

,start,end,interval_count
chrom,,,
chr1,0,5,1
chr2,0,5,4
chr2,5,10,7
chr2,10,15,2
chr2,15,20,0
chr2,20,25,0
chr2,25,30,0
chr2,30,35,0
chr2,35,40,0


You can further convert the index to colums like this:

In [93]:
df.reset_index(drop=True, level=-1).reset_index()

,chrom,start,end,interval_count
0,chr1,0,5,1
1,chr2,0,5,4
2,chr2,5,10,7
3,chr2,10,15,2
4,chr2,15,20,0
5,chr2,20,25,0
6,chr2,25,30,0
7,chr2,30,35,0
8,chr2,35,40,0
9,chr2,40,45,1


You can group by more than just the chromosome if you like:

In [95]:
(data.groupby(['chrom', 'species'])
 .apply(interval_count)
 .reset_index(drop=True, level=-1)
)

start  end  interval_count
chrom species                            
chr1  7            0    5               1
chr2  7            0    5               4
      7            5   10               7
      7           10   15               2
      7           15   20               0
      7           20   25               0
      7           25   30               0
      7           30   35               0
      7           35   40               0
      7           40   45               1

You can use hte `even` keyword to put approximately the same amount of interval in each window (to the extent that this is possible):

In [97]:
@window(even=15)
def interval_sum(df):
    return (df.end-df.start).sum()

(data.groupby('chrom')
 .apply(interval_count)
 .reset_index(drop=True, level=-1)
)

,start,end,interval_count
chrom,,,
chr1,0,5,1
chr2,0,5,4
chr2,5,10,7
chr2,10,15,2
chr2,15,20,0
chr2,20,25,0
chr2,25,30,0
chr2,30,35,0
chr2,35,40,0


You can return any number of values from your function. Just do so as a dictionary:

In [105]:
@window(even=5)
def multiple_stats(df):
    return dict(tot_length=(df.end-df.start).sum(), interval_count=len(df))
    
(data.groupby(['chrom'])
 .apply(multiple_stats)
 .reset_index(drop=True, level=-1)
)

,start,end,tot_length,interval_count
chrom,,,,
chr1,0,5,5,1
chr2,0,5,6,4
chr2,5,7,6,4
chr2,7,9,6,4
chr2,9,11,5,3
chr2,11,43,5,2
chr2,43,45,2,1


In [ ]:
If some of the windows are empty, your function must be ablte to handle that:

In [124]:
pd.DataFrame(columns=data.columns)

,chrom,start,end,value,species,run,analysis


# MAYBE MAKE A NEW VERSION FO THE REPOSITOTY:

## MAYBE MAKE EMPTY DATA FRAMES HAVE THE PROPER COLUMN NAMES SO EMPTY DATAFRAMES ARE MORE EASILY HANDLE. 

In [121]:
@window(size=10)
def stats_fun(df):
    if df.empty:
        print(df.columns)
        return 1

    return df.species.sum()
#     sr = df[['species','run']].sum()
#     return sr.to_dict()

(data.groupby(['chrom'])
 .apply(stats_fun)
 .reset_index(drop=True, level=-1)
)

Index([], dtype='object')
Index([], dtype='object')


,start,end,stats_fun
chrom,,,
chr1,0,10,7
chr2,0,10,63
chr2,10,20,14
chr2,20,30,1
chr2,30,40,1
chr2,40,50,7


In [99]:
@window(size=100000000, empty=True, fill='hg19')
def count1(df):
    return len(df.index)

(data.groupby('chrom')
 .apply(count1)
 .reset_index(drop=True, level=-1)
)

,start,end,count1
chrom,,,
chr1,0,100000000,1
chr1,100000000,200000000,0
chr1,200000000,300000000,0
chr2,0,100000000,10
chr2,100000000,200000000,0
chr2,200000000,300000000,0


In [100]:
# call this function on windows beginning at size 2 increasing by log 2
@window(size=2, logbase=2)
def count2(df):
    return len(df.index)

(data.groupby('chrom')
 .apply(count2)
 .reset_index(drop=True, level=-1)
)

,start,end,count2
chrom,,,
chr1,0.0,2.0,1
chr1,2.0,6.0,1
chr2,0.0,2.0,1
chr2,2.0,6.0,4
chr2,6.0,14.0,6
chr2,14.0,30.0,0
chr2,30.0,62.0,1


In [101]:
# call this function on windows with ~10 observations in each
@window(even=10)
def count3(df):
    return {'count': len(df.index), 'sum': sum(df.end-df.start)}

(data.groupby('chrom')
 .apply(count3)
 .reset_index(drop=True, level=-1)
)

,start,end,count,sum
chrom,,,,
chr1,0,5,1,5
chr2,0,7,6,12
chr2,7,11,5,11
chr2,11,45,2,7
